Search Engine with Toolsa and Agent

In [24]:
##Arxiv-- Research
##Tools creation
#AIP warapper for rapping tools WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [25]:
#wrapper to use wikipedia API to conduct search and fetch page summaries
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

#to run the wrapper
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [26]:
#for Arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [27]:
#to combine both
tools = [wiki,arxiv]

In [28]:
#creating custom tools rag tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [29]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(document,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8f11335000>)

Now inorder to use this as a retriever as a tool, we need to convert it to tool 

In [30]:
#to convert
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search-tool",'search any information about lang smith')

In [31]:
retriever_tool.name

'langsmith-search-tool'

In [32]:
tools =[wiki,arxiv,retriever_tool]

#runing this tools with agents and llm model

##tools, llm --> Agent

In [33]:
from dotenv import load_dotenv
import os
import openai
from langchain_groq import ChatGroq

load_dotenv()
groq_api_key = os.getenv('GROQ_API')
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key,model_name = 'llama-3.1-8b-instant')

In [34]:
#prompt template using langchain HUB
from langchain import hub
prompt =hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [35]:
#creating Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [36]:
#run the agent using agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent,tools = tools,verbose=True)
agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [37]:
#executing agent executor
agent_executor.invoke({"input":"Tell me about langsmith search"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search-tool` with `{'query': 'langsmith search'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log y

{'input': 'Tell me about langsmith search',
 'output': 'The LangSmith platform is used for building production-grade LLM applications. It allows for closely monitoring and evaluating applications, enabling quick and confident shipping. LangSmith works independently of LangChain. To get started, you need to install LangSmith, create an API key, set up your environment, and log your first trace. After that, you can run your first evaluation.'}

In [38]:
#executing agent executor
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search-tool` with `{'query': 'machine learning'}`


= traceable(async (user_input) => {    const result = await client.chat.completions.create({        messages: [{ role: "user", content: user_input }],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation​Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"

{'input': 'What is machine learning',
 'output': 'Machine learning is a field of study that focuses on the use of algorithms and statistical models to enable machines to perform a specific task, such as image or speech recognition, or decision-making, without being explicitly programmed.'}

In [39]:
#executing agent executor
agent_executor.invoke({"input":"what is this research paper all about: arXiv:1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'arXiv:1706.03762'}`


No good Arxiv Result was found
Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper "Attention Is All You Need" (arXiv:1706.03762) is a seminal paper that introduces the Transformer model, a type of neural network architecture that revolutionized the field of natural language processing. The paper, published in 2017, describes the Transformer's ability to process sequences of input data in parallel, without the need for recursive neural networks or convolutional neural networks. This innovation enabled the development of more efficient and effective language models, which have since become a cornerstone of many NLP applications

{'input': 'what is this research paper all about: arXiv:1706.03762',
 'output': 'The research paper "Attention Is All You Need" (arXiv:1706.03762) is a seminal paper that introduces the Transformer model, a type of neural network architecture that revolutionized the field of natural language processing. The paper, published in 2017, describes the Transformer\'s ability to process sequences of input data in parallel, without the need for recursive neural networks or convolutional neural networks. This innovation enabled the development of more efficient and effective language models, which have since become a cornerstone of many NLP applications.'}